In [9]:
import os
import pandas as pd
import re
import numpy as np
from rapidfuzz import fuzz, process
from user_agents import parse
from deviceMatching.findData import *

In [10]:
# Specify the directory containing the datasets
data_folder = "./data/"

#Loading in device dictionary (Already cleaned)
device_dictionary = pd.read_csv(find_file('final_device.csv', data_folder))
dirty_dictionary = pd.read_csv(find_file('final_test.csv', data_folder))

#Loading in log data (sample data)
log_data = pd.read_csv(find_file('sample_data.csv', data_folder))

file_path = "./data/intern/sample/log_devices_fo_app.parquet"
log_data_parquet = pd.read_parquet(file_path)


In [11]:
display(log_data_parquet)
display(log_data)

,user_id,user_agent,brand,model,marketing_name,os,os_version,browser,device_id,device_type,is_mobile,platform,d
0,1002671200,International/1.6.2 (iPhone; iOS 16.7.10; Scal...,Apple,iPhone,None,iOS,16.7.10,Mobile Safari UI/WKWebView,None,mobile,True,app,2024-12-01
1,1033612347,International/1.6.2 (iPhone; iOS 18.1.1; Scale...,Apple,iPhone,None,iOS,18.1.1,Mobile Safari UI/WKWebView,None,mobile,True,app,2024-12-01
2,None,NGOISAO/1.5.0 (iPhone; iOS 18.1; Scale/3.00),Apple,None,None,iOS,%2018.1,Mobile Safari UI/WKWebView,A9D4CB11-9028-4613-9561-9053014D4E2A,mobile,True,app,2024-12-01
3,None,VnExpress/7.3.5 (iPad; iOS 14.2; Scale/2.00),Apple,None,None,iOS,%2014.2,VnExpress,961670BA-82E0-4E0B-975B-43BCAE870655,mobile,True,app,2024-12-01
4,None,VnExpress/7.5.12 (iPhone; iOS 17.6.1; Scale/3.00),Apple,None,None,iOS,%2017.6.1,VnExpress,45CEF440-5386-4361-83E9-FE3C6245CB2E,mobile,True,app,2024-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20807427,1102317691,None,Apple,None,None,iOS,%2018.1.1,VnExpress,None,mobile,True,web,2025-01-06
20807428,1091266940,None,Samsung,SM-G988N,Galaxy S20 Ultra 5G,Android,13,Android,None,mobile,True,web,2025-01-06
20807429,1057166695,None,None,21081111RG,None,Android,14,Android,None,mobile,True,web,2025-01-06
20807430,1061484653,None,None,2107113SG,None,Android,11,Android,None,mobile,True,web,2025-01-06


,device_id,user_agent,mac,brand,model,marketing_name,os,device_type,source
0,6a34f3ba8ad2dfa3,Dalvik/2.1.0 (Linux; U; Android 6.0.1; FPT Pla...,FC:D5:D9:1B:CF:6A,Amlogic,FPT Play Box,NaN,android,box,FPlay
1,51a85fd7085fd761,okhttp/3.10.0,FC:D5:D9:28:05:57,SDMC,FPT ATV Play Box,NaN,other,box,FPlay
2,5c4f801a6f13548a,Dalvik/2.1.0 (Linux; U; Android 6.0.1; FPT Pla...,FC:D5:D9:15:01:AA,Amlogic,FPT Play Box,NaN,android,box,FPlay
3,477255adbcc83849,Dalvik/2.1.0 (Linux; U; Android 9; FPT ATV Pla...,FC:D5:D9:38:08:99,SDMC,FPT ATV Play Box,NaN,android,box,FPlay
4,e307f43fec1838f5,Dalvik/2.1.0 (Linux; U; Android 9; FPT ATV Pla...,FC:D5:D9:4B:77:AC,SDMC,FPT ATV Play Box,NaN,android,box,FPlay
...,...,...,...,...,...,...,...,...,...
595,NaN,Mozilla/5.0 (iPhone; CPU iPhone OS 12_5_7 like...,NaN,NaN,NaN,NaN,NaN,NaN,VNE (log Web)
596,NaN,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,NaN,NaN,NaN,NaN,NaN,NaN,VNE (log Web)
597,NaN,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...,NaN,NaN,NaN,NaN,NaN,NaN,VNE (log Web)
598,NaN,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...,NaN,NaN,NaN,NaN,NaN,NaN,VNE (log Web)


In [13]:
def identify_categorical_columns(df):
    """
    Identifies which columns in a DataFrame have a categorical constraint.

    Parameters:
    df (pd.DataFrame): The input DataFrame.

    Returns:
    list: A list of column names with categorical constraints.
    """
    # Use Pandas select_dtypes to find columns of type 'category'
    categorical_columns = df.select_dtypes(include=['category']).columns.tolist()

    return categorical_columns

categorical_cols = identify_categorical_columns(log_data_parquet)

In [14]:
display(categorical_cols)

['d']

In [15]:
def log_data_clean(df):
    """
    Cleans a dataset by:
    - Replacing NaN values with "unknown".
    - Converting all text columns to lowercase, except for 'user_agent' and 'mac'.

    Parameters:
    df (pd.DataFrame): The input dataframe.

    Returns:
    pd.DataFrame: A new cleaned dataframe (original remains unchanged).
    """

    # Make a copy to avoid modifying the original dataframe1
    df_cleaned = df.copy()

    # Replace NaN values with "unknown"
    df_cleaned.fillna("unknown", inplace=True)

    # List of columns to exclude
    exclude_columns = ['user_agent', 'mac']

    # Loop through all columns and convert to lowercase if not in exclude list
    for col in df_cleaned.columns:
        if col not in exclude_columns:
            df_cleaned[col] = df_cleaned[col].astype(str).str.lower()

    return df_cleaned

log_data_proccessed = log_data_clean(log_data)
display(log_data_proccessed)


,device_id,user_agent,mac,brand,model,marketing_name,os,device_type,source
0,6a34f3ba8ad2dfa3,Dalvik/2.1.0 (Linux; U; Android 6.0.1; FPT Pla...,FC:D5:D9:1B:CF:6A,amlogic,fpt play box,unknown,android,box,fplay
1,51a85fd7085fd761,okhttp/3.10.0,FC:D5:D9:28:05:57,sdmc,fpt atv play box,unknown,other,box,fplay
2,5c4f801a6f13548a,Dalvik/2.1.0 (Linux; U; Android 6.0.1; FPT Pla...,FC:D5:D9:15:01:AA,amlogic,fpt play box,unknown,android,box,fplay
3,477255adbcc83849,Dalvik/2.1.0 (Linux; U; Android 9; FPT ATV Pla...,FC:D5:D9:38:08:99,sdmc,fpt atv play box,unknown,android,box,fplay
4,e307f43fec1838f5,Dalvik/2.1.0 (Linux; U; Android 9; FPT ATV Pla...,FC:D5:D9:4B:77:AC,sdmc,fpt atv play box,unknown,android,box,fplay
...,...,...,...,...,...,...,...,...,...
595,unknown,Mozilla/5.0 (iPhone; CPU iPhone OS 12_5_7 like...,unknown,unknown,unknown,unknown,unknown,unknown,vne (log web)
596,unknown,Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_1 like...,unknown,unknown,unknown,unknown,unknown,unknown,vne (log web)
597,unknown,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...,unknown,unknown,unknown,unknown,unknown,unknown,vne (log web)
598,unknown,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...,unknown,unknown,unknown,unknown,unknown,unknown,vne (log web)


In [16]:
# Exclude column "d" and pass the modified DataFrame to the function
log_data_parquet_processed = log_data_clean(log_data_parquet.loc[:, log_data_parquet.columns != "d"])


In [17]:
from rapidfuzz import process, fuzz

def filter_by_brand(log_brand: str, device_dictionary: pd.DataFrame, fuzzy_threshold: int = 80) -> pd.DataFrame:
    """
    Filters the device dictionary to match the brand from the log entry.

    Args:
        log_brand (str): The brand from the log entry.
        device_dictionary (pd.DataFrame): The device dictionary DataFrame with a 'Brand' column.
        fuzzy_threshold (int): The minimum similarity score for fuzzy matching.

    Returns:
        pd.DataFrame: Subset of the device dictionary matching the brand.
    """
    if not log_brand or not isinstance(log_brand, str):
        return pd.DataFrame()  # Return empty DataFrame for invalid input

    # Normalize log brand
    log_brand_normalized = log_brand.lower().strip()

    # Step 1: Exact Matching
    exact_match = device_dictionary[
        device_dictionary["Brand"].str.lower().str.strip() == log_brand_normalized
    ]
    if not exact_match.empty:
        return exact_match

    # Step 2: Fuzzy Matching
    # Extract all unique brands from the dictionary
    unique_brands = device_dictionary["Brand"].str.lower().str.strip().unique()

    # Perform fuzzy matching
    match, score, _ = process.extractOne(log_brand_normalized, unique_brands)

    # Check if the best match meets the threshold
    if score >= fuzzy_threshold:
        # Return rows matching the best fuzzy match
        fuzzy_match = device_dictionary[
            device_dictionary["Brand"].str.lower().str.strip() == match
        ]
        return fuzzy_match

    # Step 3: No Match Found
    return pd.DataFrame()  # Return empty DataFrame if no match is found

In [18]:
def filter_by_model_exact(log_model: str, log_marketing_name: str, device_dictionary: pd.DataFrame) -> dict:
    """
    Filters the device dictionary to find an exact match for the given log model or marketing name.

    Args:
        log_model (str): The model from the log entry.
        log_marketing_name (str): The marketing name from the log entry.
        device_dictionary (pd.DataFrame): The device dictionary containing Model Name and Models.

    Returns:
        dict: Best match details or None if no exact match is found.
    """
    # # Normalize input
    # log_model = log_model.lower().strip()
    # log_marketing_name = log_marketing_name.lower().strip()

    # Case 1: Both are "unknown"
    if log_model == "unknown" and log_marketing_name == "unknown":
        return {"matched_model_name": None, "matched_models": None, "source": "No Match"}

    # Case 2: Only one value is "unknown"
    if log_model == "unknown":
        log_model = log_marketing_name
    elif log_marketing_name == "unknown":
        log_marketing_name = log_model

    # Case 3: Check for exact matches
    for search_term, column_name in [
        (log_model, "Model Name"),
        (log_marketing_name, "Model Name"),
        (log_model, "Models"),
        (log_marketing_name, "Models"),
    ]:
        exact_match = device_dictionary[
           device_dictionary[column_name].str.lower().str.strip().str.contains(search_term) #Make contains, therefore this might only work when brand subset is completed
        ]
        if not exact_match.empty:
            return {
                "matched_model_name": exact_match.iloc[0]["Model Name"],
                "matched_models": exact_match.iloc[0]["Models"],
                "source": f"{column_name} (Exact)"
            }

    # No match found
    return {"matched_model_name": None, "matched_models": None, "source": "No Match"}


In [19]:
log_data_parquet_processed_sample = log_data_parquet_processed.sample(10000)

In [20]:
# Lists to store matched and unmatched entries
matched_data = []
unmatched_data = []

# Iterate through log_data_processed and attempt to match
index = 0
for _, log_entry in log_data_parquet_processed_sample.iterrows():
    log_model = log_entry["model"]
    log_marketing_name = log_entry["marketing_name"]
    log_brand = log_entry["brand"]

    brand_subset = filter_by_brand(log_brand, dirty_dictionary)
    
    if brand_subset.empty: 
    # Call the filter_by_model function
        match_result = filter_by_model_exact(log_model, log_marketing_name, dirty_dictionary)
    else:
        match_result = filter_by_model_exact(log_model, log_marketing_name, brand_subset)

    # Organize results into matched or unmatcahed
    if match_result["matched_model_name"] is not None:
        matched_data.append({
            "log_device_id": log_entry["device_id"],
            "log_brand": log_entry["brand"],
            "log_model": log_model,
            "log_marketing_name": log_marketing_name,
            "matched_model_name": match_result["matched_model_name"],
            "matched_models": match_result["matched_models"],
            "source": match_result["source"]
        })
    else:
        unmatched_data.append({
            "log_device_id": log_entry["device_id"],
            "log_brand": log_entry["brand"],
            "log_model": log_model,
            "log_marketing_name": log_marketing_name
        })

# Convert lists to DataFrames for better readability
matched_df = pd.DataFrame(matched_data)
unmatched_df = pd.DataFrame(unmatched_data)


/tmp/ipykernel_39556/3770765697.py:35: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  device_dictionary[column_name].str.lower().str.strip().str.contains(search_term) #Make contains, therefore this might only work when brand subset is completed
/tmp/ipykernel_39556/3770765697.py:35: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  device_dictionary[column_name].str.lower().str.strip().str.contains(search_term) #Make contains, therefore this might only work when brand subset is completed
/tmp/ipykernel_39556/3770765697.py:35: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  device_dictionary[column_name].str.lower().str.strip().str.contains(search_term) #Make contains, therefore this might only work when brand subset is completed
/

In [21]:
# Filter unmatched entries where log_brand and log_model are not both "unknown"
filtered_unmatched_df = unmatched_df[
    ~((unmatched_df["log_model"].str.lower() == "unknown") & (unmatched_df["log_marketing_name"].str.lower() == "unknown"))
]

unknown = unmatched_df[((unmatched_df["log_model"].str.lower() == "unknown") & (unmatched_df["log_marketing_name"].str.lower() == "unknown"))]

samsung_unmatched_df = filtered_unmatched_df[filtered_unmatched_df["log_brand"].str.lower() == "samsung"]

print("Unmatched Entried")
display(unmatched_df)

print("Filtered Unmatched Entries (Excluding Both 'Unknown'):")
display(filtered_unmatched_df)

print("matched Entries:")
display(matched_df)

print("Display Unknown Entries")
display(unknown)

Unmatched Entried


,log_device_id,log_brand,log_model,log_marketing_name
0,unknown,unknown,23021raaeg,unknown
1,6d8ece3a-c616-4101-91a5-38de14892407,apple,unknown,unknown
2,unknown,unknown,unknown,unknown
3,unknown,apple,"iphone10,2/iphone12,3/iphone11,2/iphone10,3",unknown
4,unknown,unknown,unknown,unknown
...,...,...,...,...
6205,e28b9eda-c80b-4e45-b76c-4b3bc63a709d,apple,unknown,unknown
6206,unknown,apple,unknown,unknown
6207,unknown,apple,unknown,unknown
6208,unknown,unknown,unknown,unknown


Filtered Unmatched Entries (Excluding Both 'Unknown'):


,log_device_id,log_brand,log_model,log_marketing_name
0,unknown,unknown,23021raaeg,unknown
3,unknown,apple,"iphone10,2/iphone12,3/iphone11,2/iphone10,3",unknown
8,8dfbb70c1e474e20bcab95f27f6742b6,unknown,2201117tg,unknown
9,unknown,apple,"iphone13,3/iphone13,2",unknown
11,unknown,unknown,23129raa4g,unknown
...,...,...,...,...
6167,unknown,unknown,sm-g990e,unknown
6168,unknown,apple,"iphone10,2/iphone12,3/iphone11,2/iphone10,3",unknown
6179,unknown,unknown,tx3mini+,unknown
6198,1add2ede0eaab97570c059bb2fe655d1,unknown,22120rn86g,unknown


matched Entries:


,log_device_id,log_brand,log_model,log_marketing_name,matched_model_name,matched_models,source
0,unknown,unknown,k,unknown,unknown,"smartfren andromax ad681h, ad681h",Model Name (Exact)
1,0785e69713cafb71a26b56f867976dea,samsung,sm-a336e,unknown,galaxy a33 5g,"sm-a336e, a33x",Models (Exact)
2,unknown,samsung,sm-m205g,galaxy m20,galaxy m20,"sm-m205n, m20",Model Name (Exact)
3,unknown,unknown,k,unknown,unknown,"smartfren andromax ad681h, ad681h",Model Name (Exact)
4,d910473451723a99b6d9627fd967b933,samsung,sm-a515f,galaxy a51,galaxy a51,"sm-a515f, a51",Model Name (Exact)
...,...,...,...,...,...,...,...
3785,unknown,unknown,k,unknown,unknown,"smartfren andromax ad681h, ad681h",Model Name (Exact)
3786,unknown,unknown,k,unknown,unknown,"smartfren andromax ad681h, ad681h",Model Name (Exact)
3787,unknown,unknown,k,unknown,unknown,"smartfren andromax ad681h, ad681h",Model Name (Exact)
3788,14f1c519388e588a5bf818d49abcef48,oppo,cph2531,unknown,reno10 5g,"cph2531, op5705l1",Models (Exact)


Display Unknown Entries


,log_device_id,log_brand,log_model,log_marketing_name
1,6d8ece3a-c616-4101-91a5-38de14892407,apple,unknown,unknown
2,unknown,unknown,unknown,unknown
4,unknown,unknown,unknown,unknown
5,24cc53a6-eeae-47f0-a9f2-2c4cfb70e7d0,apple,unknown,unknown
6,unknown,unknown,unknown,unknown
...,...,...,...,...
6204,536f22fe-1ea3-4f8d-9b83-e1a9a55fc708,apple,unknown,unknown
6205,e28b9eda-c80b-4e45-b76c-4b3bc63a709d,apple,unknown,unknown
6206,unknown,apple,unknown,unknown
6207,unknown,apple,unknown,unknown


In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(filtered_unmatched_df)

,log_device_id,log_brand,log_model,log_marketing_name
11,unknown,unknown,21061119ag,unknown
12,unknown,apple,"iphone13,3/iphone13,2",unknown
16,unknown,apple,"ipad7,11/ipad7,12",unknown
18,unknown,apple,"ipad11,3/ipad11,4",unknown
25,unknown,unknown,sm-a346e,unknown
27,unknown,unknown,g11,unknown
37,unknown,unknown,sm-a135f,unknown
38,unknown,apple,"iphone10,2/iphone12,3/iphone11,2/iphone10,3",unknown
51,unknown,apple,"iphone12,5/iphone11,4/iphone12,1/iphone11,8",unknown
61,unknown,unknown,sm-s918b,unknown


Problem is from the brand subset. 